# Synopsis

So far we've covered the basics of what constitutes an image, color is encoded, and how we can manipulate it. However, when you conduct research the tasks you need to perform are typically more complex (although they always seem easy to do before you start coding!). Some examples of common tasks are:

* Automatically identify regions
* Identify the borders of said regions
* Find bright spots/blobs
* Skeletonize shapes (i.e. find the backbone)

We'll go over some basic methods to do some of these methods using [`scikit-image`](https://scikit-image.org/). The  package is a sister to the `scikit-learn` package, both of these packages are focused on implementing machine-learning methods in Python but `scikit-image`, as you probably guessed, is geared towards algorithms that can be applied to images. 

`scikit-image` functions are stored in the library `skimage`.

# Read libraries

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from colorama import Back, Fore, Style
from pathlib import Path
from sys import path

path.append( str(Path.cwd().parent) )
path

In [ ]:
my_fontsize = 15

In [ ]:
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

from pylab import imread, imshow
from skimage import data, img_as_float, measure, transform

from Amaral_libraries.my_stats import half_frame

# Identifying image foreground features

One common technique researchers will want to use is identifying the edge of certain regions. This is useful in a number of contexts, such as:

* Identifying the borders of cells in a microscope image
* Finding areas in geographic areas

This procedure is typically called contour finding and we can use an algorithm that is implemented in `scikit-image`.

In [ ]:
coins_original = img_as_float( data.coins() )

print(Style.BRIGHT, 'Shape:', Style.RESET_ALL, coins_original.shape)
print()

print(f" Maximum in image is {coins_original.max():.3f}, minimum is "
      f"{coins_original.min():.3f}.\n")


imshow(coins_original, cmap = 'gray', vmin = 0, vmax = 1);


## Zooming in

To help us with image processing it is often useful to be able to magnify parts of an image.  The function below does exactly that for grayscale images.

As an exercise, create a similar function for magnifying portions of `RGB` images.

In [ ]:
def get_slice_coordinates(x, view_side, w):
    """
    This function returns the viewing coordinates along an axis
    given a viewing center, a view width, and an image length 
    along axis.
    
    inputs:
        x -- int viewing center
        view_side -- int view width
        w -- int image length
        
    returns
        x0 -- int
        x1 -- int
    """
    
    if int(0.5*view_side) > x:
        x0 = 0
        x1 = view_side
        
    elif int(0.5*view_side + x) >= w :
        x1 = w
        x0 = w - view_side
        
    else:
        x0 = x - (0.5 * view_side)
        x1 = x0 + view_side

    return int(x0), int(x1)


def flat_zoom_at(image, x, y, linear_zoom):
    """
    This function zooms in on position x, y of image at a 
    magnification of linear_zoom
    
    inputs:
        image -- array
        x -- int viewing center
        y -- int viewing center
        linear_zoom -- int
        
    returns:
        new_image -- array
        x0 -- int view start coordinates
        y0 -- int view start coordinates
    
    """
    h, w = image.shape
    view_side = int( min(w, h) / linear_zoom )
    
    x0, x1 = get_slice_coordinates(x, view_side, w)
    y0, y1 = get_slice_coordinates(y, view_side, h)
        
    new_image = transform.rescale( image[ y0 : y1, x0 : x1], linear_zoom )

    return new_image, x0, y0
    

In [ ]:
linear_zoom = 4
x = 43
y = 51
new_image, x0, y0 = flat_zoom_at(coins_original, x, y, linear_zoom)

fig = plt.figure( figsize = (10, 6))
ax = []

ax.append(fig.add_subplot(121))
ax[-1].imshow( coins_original[:150, :150], cmap = 'gray', 
               vmin = 0, vmax = 1 )
ax[-1].plot([x], [y], 'ro');

ax.append(fig.add_subplot(122))
ax[-1].imshow( new_image, cmap = 'gray', vmin = 0, vmax = 1 )
ax[-1].plot([linear_zoom*(x-x0)], [linear_zoom*(y-y0)], 'ro');

plt.tight_layout()

## Contours

Next we can use a contour finding algorithm. In `scikit-image` there is the `find_contours` algorithm that is an implementation of the [marching squares algorithm](http://users.polytech.unice.fr/~lingrand/MarchingCubes/algo.html). This algorithm constructs the image as a grid and attempts to draw lines along the edges of the squares in the grid. 

In [ ]:
contours = measure.find_contours(coins_original)
print(f"The method .find_contours returns a {type(contours)}.\n"
      f"Each contour is an array.\n")

print(contours[0])


print(f"\n\n--> The algorithm found {len(contours)} contours even "
      f"though there\nare only 24 coins in the image.\n" )




That seems a bit much...  So let us see what this did... 

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

ax.imshow(coins_original, cmap = 'gray', vmin = 0, vmax = 1)

for n, contour in enumerate(contours):
    ax.plot(contour[:, 1], contour[:, 0], linewidth=2)


**Holy &%(^&!!!!**

Yes, there are some good contours there but...

If you look at the image attentively, it becomes clear that the background in the top left corner is brighter than in the bottom right corner.

Moreover, no contours are found in the top left corner, whereas many absurd looking contours are found in the transition region between dark and light background. 

The `find_countours()` function took two arguments from us: image array and a `level` parameter value. 

> Parameters
>
> ----------
>
> image : 2D ndarray of double
>
>    Input image in which to find contours.
>
> level : float, optional
> 
> Value along which to find contours in the array. By default, the level
>    is set to (max(image) + min(image)) / 2


The `level` controls the value around which the algorithm should attempt to find the contours - this is our free parameter.  We set it to 0.2, which is not really great. 

But, **is there a good value?**

Let's play with it.

In [ ]:
#Try level from 0.2 to 0.8
#
level = 0.2
minimum_contour_len = 40 

fig = plt.figure()
ax = fig.add_subplot(111)

ax.imshow(coins_original, cmap = 'gray', vmin = 0, vmax = 1)

contours = measure.find_contours(coins_original, level)
for n, contour in enumerate(contours):
    
    # We exclude very short contours, which likely just noise
    #
    if len(contour) > minimum_contour_len:
        ax.plot(contour[:, 1], contour[:, 0], linewidth=2)

## Identifying the background

Clearly, there is no good single value. The problem is that the background has different properties in different regions of the image $-$ compare background inside and outside the **very long contour**. 

So maybe the solution is to try to make the background uniform...

But first we need to find out what the background properties are...


In [ ]:
w, h = coins_original.shape
intensities = coins_original.reshape((w*h, 1)) # Make it 1D for plotting

print(f"There are {len(intensities[:,0])} pixels in the image.\n")
print(f"Their average intensity is {np.median(intensities):.3f} "
      f"(theoretical maximum is 1).\n")

fig = plt.figure(figsize = (10, 4))
ax = fig.add_subplot(111)

half_frame(ax, 'Intensity', 'Frequency', font_size = my_fontsize)
ax.hist(intensities, bins = np.arange(0, 1, 0.02), rwidth = 0.9)

ax.vlines(np.median(intensities), 0, 6000, color = 'black', lw = 4)

plt.tight_layout()

**About 50% of image is background, so we could set all values below the median intensity to zero and see what that does to the image...**


In [ ]:
image_mask = coins_original > np.median(intensities)

fig = plt.figure(figsize = (12, 4))
ax = []

ax.append( fig.add_subplot(121) )
ax[-1].imshow(coins_original * image_mask, cmap = 'gray', 
              vmin = 0, vmax = 1)


ax.append( fig.add_subplot(122) )
ax[-1].imshow(coins_original, cmap = 'gray', vmin = 0, vmax = 1)

plt.tight_layout()

It is visually apparent that some of the background is brighter than the median intensity and that some pixels within the coins are darker.

In [ ]:
print(Style.BRIGHT, f"{'Original':>20} {'With mask':>35}", Style.RESET_ALL)

fig = plt.figure( figsize = (8, 8))
ax = []

ax.append(fig.add_subplot(221))
new_image, x0, y0 = flat_zoom_at(coins_original, 50, 50, 4)
ax[-1].imshow(new_image, cmap = 'gray', vmin = 0, vmax = 1)

ax.append(fig.add_subplot(222))
new_image, x0, y0 = flat_zoom_at(coins_original * image_mask, 50, 50, 4)
ax[-1].imshow(new_image, cmap = 'gray', vmin = 0, vmax = 1)

ax.append(fig.add_subplot(223))
new_image, x0, y0 = flat_zoom_at(coins_original, 200, 275, 4)
ax[-1].imshow(new_image, cmap = 'gray', vmin = 0, vmax = 1)

ax.append(fig.add_subplot(224))
new_image, x0, y0 = flat_zoom_at(coins_original * image_mask, 200, 275, 4)
ax[-1].imshow(new_image, cmap = 'gray', vmin = 0, vmax = 1)

plt.tight_layout()

### Backgrounds with changing properties

So, it is not great that some regions insider the foreground were set to zero or that the background in the top left corner remained unchanged.

A solution to this issue is to set a threshold for the background that depends on the specific region of the image

In [ ]:
# divide figure into n^2 sections
n = 4

step_x = int( (w / n) + 0.5)
step_y = int( (h / n) + 0.5)
section = []

for i in range(n):
    section.append([])
    for j in range(n):
        section[-1].append(coins_original[i*step_x:(i+1)*step_x,
                                          j*step_y:(j+1)*step_y])

for i in range(n):
    for j in range(n):
        w_1, h_1 = section[i][j].shape
        section_intensities = section[i][j].reshape((w_1*h_1,1))
        section_quantile = np.quantile(section_intensities, 0.4)
#         print(section_intensities.shape, section_quantile)
        mask = section[i][j] > max(0.33, section_quantile)
        section[i][j] = section[i][j] * mask
        imshow(section[i][j], cmap = 'gray', vmin = 0, vmax = 1)
        plt.show()

In [ ]:
coins_filtered = []
for i in range(n):
    coins_filtered.append( np.concatenate(section[i][:], axis = 1) )
                          
coins_filtered = np.concatenate(coins_filtered, axis = 0)

imshow(coins_filtered, cmap = 'gray', vmin = 0, vmax = 1);


## Contour length as a clue to relevance

Considering the contours identified earlier, it is clearly that not all of them are similar. Some are very small and likely are identifying similar regions within the background or regions of the foreground. Others are very long and may be related to a background with changing properties.

In [ ]:
level = 0.4

fig = plt.figure( figsize = (12, 6))
ax1 = fig.add_subplot(122)
ax2 = fig.add_subplot(121)

ax1.imshow(coins_original, cmap = 'gray', vmin = 0, vmax = 1)
ax2.imshow(coins_filtered, cmap = 'gray', vmin = 0, vmax = 1)

contours = measure.find_contours(coins_filtered, level)
clean_contours = []
for n, contour in enumerate(contours):
    if len(contour) > 100 and len(contour) < 500:
        clean_contours.append(contour)
        ax1.plot(contour[:, 1], contour[:, 0], linewidth = 1)
        ax2.plot(contour[:, 1], contour[:, 0], linewidth = 2)
        
print(f"We got {len(clean_contours)} clean contours for our 24 coins.")

Pretty nice, ah?

## Properties of foreground features

Now that we have our contours, if we could retrieve the pixels inside each of the contours we could calculate some properties of the objects inside.

Fortunately, there is a function that returns a mask array for points inside a polygon (or a contour):

> `measure.grid_points_in_poly`( shape, contour )



In [ ]:
contour_mask = measure.grid_points_in_poly( coins_original.shape, 
                                            clean_contours[0] )

masked_coin = coins_original * contour_mask
new_image2, x0, y0 = flat_zoom_at(masked_coin, 330, 45, 4)

fig = plt.figure( figsize = (12, 6))
ax = []

ax.append( fig.add_subplot(121) )
ax[-1].imshow( masked_coin, cmap = 'gray', vmin = 0, vmax = 1 )

ax.append( fig.add_subplot(122) )
ax[-1].imshow( new_image2, cmap = 'gray', vmin = 0, vmax = 1 )

plt.tight_layout()

By counting the number of values that are `True` in the mask, we can determine the area of our object.

We can also extract the intensities of the image inside the contour and find their distribution.


In [ ]:
area = sum( contour_mask.reshape((w*h, 1)) )
print(f"The area inside the contour is {area[0]} pixels.\n" )

intensities = list( (masked_coin).reshape((w*h, 1)) )
intensities = np.array( [x for x in intensities if x > 0] )

fig = plt.figure(figsize = (12, 4))
ax = fig.add_subplot(111)

half_frame(ax, 'Intensity', 'Frequency', font_size = my_fontsize)
ax.hist(intensities, bins = np.arange(0, 1, 0.02), rwidth = 0.9)
ax.set_xlim(0, 1)

ax.vlines(np.median(intensities), 0, 300, color = 'black', lw = 4)

plt.tight_layout()

# Identifying cells in microscopy images


Microscopy offers incredible windows into biological systems at the cellular and molecular level.  To experience this wonderful world, we will look at images of cells on plates from Cell Image Library.

We will write code to identify the contour of the cells, and then measure how well our code performs.

In [ ]:
cells_folder = Path.cwd() / 'Data' / 'Cell_images' / 'BBBC022_v1_images_20585w1'
os.listdir(cells_folder)

In [ ]:
cell_images = list( cells_folder.glob('*') )
print(len(cell_images))
print()

plate_1 = imread(cell_images[1])
print(f"Image has shape {plate_1.shape}.\n")
      
imshow(plate_1, cmap = 'gray')
plt.show()

This image is actually nicer than the one with the coins.  The background appears to be much more uniform and more distinct from the foreground.

## Function for calculating histogram of intensities

Let's put the code that calculates the distribution of intensities in a picture into a function and use it to generate histograms for all cell plates.


## Functions for identifying relevant contours

Re-write some of the code above in order to identify the contours of the cells in the plates.  Does the code need to be different based on whether the background is uniform or not?

## Functions for calculating properties of the cells

Write functions that generate a mask for the pixels within a contour.

Write a function that calculates the total number of pixels within a contour.

Write a function that calculates and histogram of the intensities within a contour and descriptive statistics of those values. 
